In [ ]:
import pandas as pd
import numpy as np
import rasterio
import joblib
from rasterio.transform import rowcol


In [ ]:
!wget https://data.worldpop.org/GIS/Population/Global_2000_2020/2020/IND/ind_ppp_2020.tif

Mounted at /content/drive


In [ ]:
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

#import data kaggle
df = pd.read_csv("pharmacies_in_west_bengal.csv")

df = df.rename(columns={
    "Latitude": "latitude",
    "Longitude": "longitude",
    "lat": "latitude",
    "lon": "longitude"
})

df = df[['latitude', 'longitude']].dropna().reset_index(drop=True)

#import data populasi
with rasterio.open("ind_ppp_2020.tif") as src:
    band = src.read(1)
    transform = src.transform

def get_population(lat, lon):
    row, col = rowcol(transform, lon, lat)
    return band[row, col]

rows, cols = rowcol(
    transform,
    df["longitude"].values,
    df["latitude"].values
)

df["population"] = band[rows, cols]




In [ ]:
#menghitung apotek dalam radius
coords = np.radians(df[['latitude', 'longitude']].values)
tree = BallTree(coords, metric="haversine")

R_EARTH = 6371  # km
r5 = 5 / R_EARTH
r10 = 10 / R_EARTH

df["pharmacy_5km"] = tree.query_radius(coords, r=r5, count_only=True)
df["pharmacy_10km"] = tree.query_radius(coords, r=r10, count_only=True)

In [ ]:
#menghitung kepdatan penduduk
area_5km = np.pi * (5 ** 2)  # km^2
df["pop_density_5km"] = df["population"] / area_5km

In [ ]:
#membuat regresi
df["need_score"] = df["pop_density_5km"] / (df["pharmacy_5km"] + 1)

In [ ]:
#training
features = ["pop_density_5km", "pharmacy_5km", "pharmacy_10km"]
X = df[features]
y = df["need_score"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

reg_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    random_state=42
)
reg_model.fit(X_scaled, y)

df["predicted_need"] = reg_model.predict(X_scaled)

In [ ]:
#melakukan klasifikasi
threshold = df["predicted_need"].quantile(0.7)
df["priority_label"] = (df["predicted_need"] >= threshold).astype(int)

In [ ]:
#train model klasifikasi
cls_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    random_state=42
)
cls_model.fit(X_scaled, df["priority_label"])

RandomForestClassifier(max_depth=10, n_estimators=300, random_state=42)

In [ ]:
#menyimpan model
joblib.dump(scaler, "scaler.pkl")
joblib.dump(reg_model, "regression_model.pkl")
joblib.dump(cls_model, "classification_model.pkl")
joblib.dump(threshold, "priority_threshold.pkl")
print("Model Telah Disimpan")



Model Telah Disimpan


In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
import folium
from folium.plugins import HeatMap

df['need_norm'] = (
    df['predicted_need'] - df['predicted_need'].min()
) / (
    df['predicted_need'].max() - df['predicted_need'].min()
)

# Base Map
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=6,
    tiles=None
)

# Base map layer
folium.TileLayer(
    tiles="CartoDB positron",
    name="Reference Map",
    control=True
).add_to(m)

# judul
title_html = """
<div style="
    position: fixed;
    top: 10px;
    left: 50%;
    transform: translateX(-50%);
    z-index: 9999;
    background: white;
    padding: 10px 20px;
    border-radius: 8px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.3);
    text-align: center;
">
    <b style="font-size:20px;">
    Population Density (PPP 2020) and Pharmacy Locations<br>
    West Bengal, India
    </b><br>
    <span style="font-size:13px;">
        Regression Heatmap & Classification Markers
    </span>
</div>
"""
m.get_root().html.add_child(folium.Element(title_html))

# heatmap klasifikasi
heat_data = df[['latitude', 'longitude', 'need_norm']].values.tolist()

HeatMap(
    heat_data,
    radius=15,
    blur=20,
    min_opacity=0.4,
    name="Predicted Need (Regression)"
).add_to(m)

# marker klasifikasi
priority_layer = folium.FeatureGroup(name="Priority Classification")

for _, row in df.iterrows():
    color = "red" if row['priority_label'] == 1 else "green"
    status = "PRIORITY" if row['priority_label'] == 1 else "NON-PRIORITY"

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=folium.Popup(
            f"""
            <b>Status:</b> {status}<br>
            <b>Predicted Need:</b> {row['predicted_need']:.2f}<br>
            <b>Population Density (5km):</b> {row['pop_density_5km']:.1f}<br>
            <b>Pharmacy 5km:</b> {row['pharmacy_5km']}
            """,
            max_width=300
        )
    ).add_to(priority_layer)

priority_layer.add_to(m)

# Legenda
legend_html = """
<div style="
    position:fixed;
    bottom: 40px;
    right: 40px;
    z-index: 9999;
    background: white;
    padding: 12px;
    border-radius: 8px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>Legend</b><br><br>

<b>Heatmap (Regression)</b><br>
<span style="
    display:inline-block;
    width:120px;
    height:10px;
    background: linear-gradient(to right, blue, cyan, yellow, red);
"></span><br>
Low &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; High
<br><br>

<b>Classification</b><br>
<i style="
    background:red;
    width:10px;
    height:10px;
    border-radius:50%;
    display:inline-block;
"></i> PRIORITY<br>

<i style="
    background:green;
    width:10px;
    height:10px;
    border-radius:50%;
    display:inline-block;
"></i> NON-PRIORITY
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Layer Control
folium.LayerControl(collapsed=False).add_to(m)

#simpan
m.save("Pharmacy Location.html")

print("SELESAI")
print("Output map: Pharmacy Location.html")


SELESAI
Output map: Pharmacy Location.html


In [ ]:
#Load Model
scaler = joblib.load("scaler.pkl")
reg_model = joblib.load("regression_model.pkl")
cls_model = joblib.load("classification_model.pkl")
threshold = joblib.load("priority_threshold.pkl")
print("Load Model")

Load Model


In [ ]:
#prediksi baru
new_data = pd.DataFrame([
    {
        "pop_density_5km": 1500,
        "pharmacy_5km": 1,
        "pharmacy_10km": 4
    },
    {
        "pop_density_5km": 300,
        "pharmacy_5km": 6,
        "pharmacy_10km": 15
    }
])

X_new_scaled = scaler.transform(new_data)

In [ ]:
#regresi baru
new_data["predicted_need"] = reg_model.predict(X_new_scaled)

#klasifikasi baru
new_data["priority_class"] = (
    new_data["predicted_need"] >= threshold
).astype(int)

print("\n HASIL PREDIKSI:")
print(new_data)


 HASIL PREDIKSI:
   pop_density_5km  pharmacy_5km  pharmacy_10km  predicted_need  \
0             1500             1              4        0.595990   
1              300             6             15        0.554828   

   priority_class  
0               1  
1               1  
